# Data Collection for Master Research

## Import the libraries and data

     --------------------------------------- 10.9/10.9 MB 10.7 MB/s eta 0:00:00
     --------------------------------------- 14.7/14.7 MB 10.9 MB/s eta 0:00:00
     -------------------------------------- 500.6/500.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import DataReader
import pandas_datareader.data as web
import requests
import json
from datetime import datetime
import re
import os
from sklearn.decomposition import PCA
pd.options.display.max_rows = 9999
pd.options.display.min_rows = None
pd.options.display.max_columns = 9999
import glob as glob
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter('ignore')

# Google Drive
path = '/content/drive/My Drive/Master_Research/workspace/data'
# # Local
# path = '../data'

# Import the user-defined modules
import sys
sys.path.append(path+'/../src')
import collecting_data
from collecting_data import *
import preprocessing_ts, conducting_EDA
from preprocessing_ts import *
from conducting_EDA import *

## Blockchain Information
Source: https://www.blockchain.com/charts

In [ ]:
blockchain_vars_list = ["total-bitcoins", "market-price", "market-cap", "trade-volume", "blocks-size", "avg-block-size", 
                        "n-transactions-per-block", "n-transactions-total", "median-confirmation-time", "avg-confirmation-time", "hash-rate",
                        "difficulty", "miners-revenue", "transaction-fees", "transaction-fees-usd", "fees-usd-per-transaction",
                        "cost-per-transaction-percent", "cost-per-transaction", "n-unique-addresses", "n-transactions", "transactions-per-second",
                        "output-volume", "mempool-count", "mempool-growth", "mempool-size", "utxo-count",
                        "n-transactions-excluding-popular", "estimated-transaction-volume", "estimated-transaction-volume-usd", "mvrv", "nvt", "nvts"]

### Original Data Range

In [ ]:
# Import all the csv files in the data folder by using glob
csvfiles = sorted(glob.glob(path+'/01_raw/blockchain/*.csv'))

# Read each csv file by pandas and make a data frame
list_blockchain = [pd.read_csv(csv_file, parse_dates=[0]) for csv_file in csvfiles]

# Change the column names
for i in range(len(list_blockchain)):
    m = re.search("blockchain/(.+?).csv", csvfiles[i])
    file_name = m.group(1)
    list_blockchain[i].columns = ["timestamp", file_name]
len(list_blockchain)

# Extract Year, Month, Day from the original info of Date
for i in range(len(list_blockchain)):
    list_blockchain[i]["Date"] = list_blockchain[i].timestamp.map(lambda x: x.strftime('%Y-%m-%d'))
    list_blockchain[i]["Date"] = pd.to_datetime(list_blockchain[i]['Date'])
    list_blockchain[i] = list_blockchain[i].set_index("Date").resample("D").mean()

# Merge all data to one dataframe
df_blockchain=list_blockchain[0]
for i in range(len(list_blockchain)):
    df_blockchain = pd.merge(df_blockchain, list_blockchain[i], on='Date', how="outer")
df_blockchain = df_blockchain.rename({"00_total-bitcoins_x": "00_total-bitcoins"},axis=1)
df_blockchain = df_blockchain.drop("00_total-bitcoins_y", axis=1)

# Make all variables numerical
df_blockchain = df_blockchain.astype(float)

df_blockchain.to_pickle(path+"/01_raw/blockchain.pickle")

### Updated



In [ ]:
# Import all the csv files in the data folder by using glob
csvfiles = sorted(glob.glob(path+'/01_raw/blockchain_updated_20210320/*.csv'))  # Change the folder name if needed: {20210211, 20210320}

# Read each csv file by pandas and make a data frame
list_blockchain = [pd.read_csv(csv_file, parse_dates=[0]) for csv_file in csvfiles]

# Change the column names
for i in range(len(list_blockchain)):
    m = re.search("blockchain_updated_20210320/(.+?).csv", csvfiles[i])  # Change the folder name if needed: {20210211, 20210320}
    file_name = m.group(1)
    list_blockchain[i].columns = ["timestamp", file_name]
len(list_blockchain)

# Extract Year, Month, Day from the original info of Date
for i in range(len(list_blockchain)):
    list_blockchain[i]["Date"] = list_blockchain[i].timestamp.map(lambda x: x.strftime('%Y-%m-%d'))
    list_blockchain[i]["Date"] = pd.to_datetime(list_blockchain[i]['Date'])
    list_blockchain[i] = list_blockchain[i].set_index("Date").resample("D").mean()

# Merge all data to one dataframe
df_blockchain_updated=list_blockchain[0]
for i in range(len(list_blockchain)):
    df_blockchain_updated = pd.merge(df_blockchain_updated, list_blockchain[i], on='Date', how="outer")
df_blockchain_updated = df_blockchain_updated.rename({"00_total-bitcoins_x": "00_total-bitcoins"},axis=1)
df_blockchain_updated = df_blockchain_updated.drop("00_total-bitcoins_y", axis=1)

# Make all variables numerical
df_blockchain_updated = df_blockchain_updated.astype(float)

df_blockchain_updated.to_pickle(path+"/01_raw/blockchain_updated_20210320.pickle")  # Change the folder name if needed: {20210211, 20210320}

# Rewrite the file to merge original and updated files
blockchain_new = pd.concat([df_blockchain, df_blockchain_updated]).groupby(level=0).last()
df_blockchain = blockchain_new.copy()

# Rename the variables
dict_convert_blockchain_vars = {
    '00_total-bitcoins': "total-bitcoins",
    '01_market-price': "btc_price",
    '02_market-cap': "market-cap",
    '03_trade-volume': "trade-volume",
    '04_blocks-size': "blocks-size",
    '05_avg-block-size': "avg-block-size",
    '06_n-transactions-per-block': "n-transactions-per-block",
    '07_n-transactions-total': "n-transactions-total",
    '08_median-confirmation-time': "median-confirmation-time",
    '09_avg-confirmation-time': "avg-confirmation-time",
    '10_hash-rate': "hash-rate",
    '11_difficulty': "difficulty",
    '12_miners-revenue': "miners-revenue",
    '13_transaction-fees': "transaction-fees",
    '14_transaction-fees-usd': "transaction-fees-usd",
    '15_fees-usd-per-transaction': "fees-usd-per-transaction",
    '16_cost-per-transaction-percent': "cost-per-transaction-percent",
    '17_cost-per-transaction': "cost-per-transaction",
    '18_n-unique-addresses': "n-unique-addresses",
    '19_n-transactions': "n-transactions",
    '20_transactions-per-second': "transactions-per-second",
    '21_output-volume': "output-volume",
    '22_mempool-count': "mempool-count",
    '23_mempool-growth': "mempool-growth",
    '24_mempool-size': "mempool-size",
    '25_utxo-count': "utxo-count",
    '26_n-transactions-excluding-popular': "n-transactions-excluding-popular",
    '27_estimated-transaction-volume': "estimated-transaction-volume",
    '28_estimated-transaction-volume-usd': "estimated-transaction-volume-usd",
    '29_mvrv': "mvrv",
    '30_nvt': "nvt",
    '31_nvts': "nvts"
    }

df_blockchain = df_blockchain.rename(dict_convert_blockchain_vars, axis=1)
df_blockchain.to_pickle(path+"/01_raw/blockchain.pickle")

### Data time-range to be collected



In [ ]:
# Time range should be equal to that of the above blockchain data
start = datetime(df_blockchain.reset_index()["Date"].dt.year[0], df_blockchain.reset_index()["Date"].dt.month[0], df_blockchain.reset_index()["Date"].dt.day[0])
end   = datetime(df_blockchain.reset_index()["Date"].dt.year[len(df_blockchain)-1], df_blockchain.reset_index()["Date"].dt.month[len(df_blockchain)-1], df_blockchain.reset_index()["Date"].dt.day[len(df_blockchain)-1])
print(start, end)

2017-09-05 00:00:00 2021-03-19 00:00:00


## Macroeconomic data
Source: https://finance.yahoo.com/

In [1]:
# # Variables and tickers
# dict_macro = {"sp500": "^GSPC", "eurostoxx": "^STOXX50E", "dow30": "^DJI", "nasdaq": "^IXIC", "oil": "CL=F", "SSE": "000001.SS", "gold": "GC=F", 
#               "vix": "^VIX", "nikkei225": "^N225", #"FTSE100": "^FTSE?P=FTSE",
#             "gbp_usd": "GBPUSD=X", "jpy_usd": "JPYUSD=X", "chf_usd": "CHFUSD=X", "cny_usd": "CNYUSD=X", "eur_usd": "EURUSD=X"}

# df_macro = yf_collect_data(dataset_name="macro", dict_ticker=dict_macro, start=start, end=end)

## Other Crypto Assets
References: 
- https://cointelegraph.com/news/does-correlation-between-bitcoin-price-and-altcoins-mean-buy-the-dips
- https://cointelegraph.com/news/truth-about-crypto-price-correlation-how-closely-does-eth-follow-btc
- https://coinpredictor.io/altcoins

-Source: https://finance.yahoo.com/u/yahoo-finance/watchlists/crypto-top-market-cap?.tsrc=fin-srch
- Top Cryptos 30 by Market Cap 

In [2]:
# # Tickers of many other crypto coins
# dict_crypto = {"ETH": "ETH-USD", "XRP": "XRP-USD", "USDT": "USDT-USD", "LTC": "LTC-USD", "BCH": "BCH-USD", "LINK": "LINK-USD", "ADA": "ADA-USD", "BNB": "BNB-USD", "DOT1": "DOT1-USD", 
#                #"DOT2": "DOT2-USD", 
#                "XLM": "XLM-USD", "USDC": "USDC-USD", "BSV": "BSV-USD", "EOS": "EOS-USD", "XMR": "XMR-USD", "XEM": "XEM-USD", "TRX": "TRX-USD", "XTZ": "XTZ-USD", "CCXX": "CCXX-USD", 
#                "NEO": "NEO-USD", "ATOM1": "ATOM1-USD", 
#                #"ATOM2": "ATOM2-USD", 
#                "VET": "VET-USD", "THETA": "THETA-USD", "IOTA": "MIOTA-USD", "ZEC": "ZEC-USD", "ETC": "ETC-USD", "WAVES": "WAVES-USD",
#                "BTC_FUTURES": "BTC=F", "CMC200": "^CMC200"}

# df_crypto = yf_collect_data(dataset_name="crypto", dict_ticker=dict_crypto, start=start, end=end)